<a href="https://colab.research.google.com/github/Toaaassem/FlexGo/blob/main/sentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# Load IMDB dataset
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

# Convert dataset to NumPy arrays
train_data, test_data = dataset["train"], dataset["test"]

train_sentences = []
train_labels = []
test_sentences = []
test_labels = []

for sentence, label in tfds.as_numpy(train_data):
    train_sentences.append(sentence.decode("utf-8"))
    train_labels.append(label)

for sentence, label in tfds.as_numpy(test_data):
    test_sentences.append(sentence.decode("utf-8"))
    test_labels.append(label)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Tokenization
vocab_size = 20000
max_length = 200
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

# Load GloVe embeddings
embedding_dim = 100  # Embedding vector size
glove_path = "https://nlp.stanford.edu/data/glove.6B.zip"
glove_file = "glove.6B.100d.txt"

# Download and extract GloVe
import os
import requests
import zipfile

if not os.path.exists("glove.6B.zip"):
    print("Downloading GloVe embeddings...")
    response = requests.get(glove_path)
    with open("glove.6B.zip", "wb") as f:
        f.write(response.content)

# Extract GloVe
if not os.path.exists(glove_file):
    with zipfile.ZipFile("glove.6B.zip", "r") as zip_ref:
        zip_ref.extractall()

embeddings_index = {}
with open(glove_file, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Build LSTM Model with Pretrained GloVe Embeddings
model = Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    Bidirectional(LSTM(128, return_sequences=True)),  # Bidirectional LSTM
    Dropout(0.3),
    LSTM(64),
    Dense(32, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Output layer for binary classification
])

# Compile model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
history = model.fit(train_padded, train_labels, validation_data=(test_padded, test_labels), epochs=5, batch_size=64)

# Evaluate model
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")

# Predict a new review
def predict_sentiment(review):
    seq = tokenizer.texts_to_sequences([review])
    padded = pad_sequences(seq, maxlen=max_length, padding="post", truncating="post")
    prediction = model.predict(padded)[0][0]
    return "Positive" if prediction > 0.5 else "Negative"

# Example usage
sample_review = "The movie was absolutely fantastic! The performances were stunning."
print(f"Review Sentiment: {predict_sentiment(sample_review)}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 671s 2s/step - accuracy: 0.5362 - loss: 0.6880 - val_accuracy: 0.5009 - val_loss: 0.6929
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 686s 2s/step - accuracy: 0.5167 - loss: 0.6918 - val_accuracy: 0.5482 - val_loss: 0.6838
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 748s 2s/step - accuracy: 0.5119 - loss: 0.6907 - val_accuracy: 0.5099 - val_loss: 0.6927
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 666s 2s/step - accuracy: 0.5246 - loss: 0.6909 - val_accuracy: 0.7336 - val_loss: 0.6097
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 698s 2s/step - accuracy: 0.7336 - loss: 0.5626 - val_accuracy: 0.8331 - val_loss: 0.3791
782/782 ━━━━━━━━━━━━━━━━━━━━ 225s 288ms/step - accuracy: 0.8348 - loss: 0.3829
Test Accuracy: 0.8331
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 776ms/step
Review Sentiment: Positive
